# Project Index

[Custom Model Notebook](../../notebooks/custom_model.ipynb)  
[Training Notebook](../../notebooks/train.ipynb)  
[Project Config Notebook](../../notebooks/project_config.ipynb)  
[Forgather Notebook](../../notebooks/forgather.ipynb)  

In [1]:
import forgather.nb.notebooks as nb

nb.display_project_index(show_pp_config=True, show_generated_code=True)

# Traning  for Fashion

This project reproduces the configuration from a PyTorch tutorial, where a simple ML model is created and trained to recognize categories of clothing from the FashionMNIST dataset.

https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

This was chosen as it is a relatively simple project which can be relativley self contained. Still, it is far more complex than the previous examples.

The model itself does not require any custom code. It's simply a stack of PyTorch layers, chained together with a nn.Sequential. If you would like to know more about the model itself, see:

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

## Custom Code

While Forgather is good at assembling objects, the language is not practical for defining logic. For this, we have defined a custom "trainer" class in the project's 'src' directory and we will use Forgather to dynamically import this code, injecting all the required dependencies.

Unlike the previous projects, you will note that the "Modules" section not empty and has a link to the Trainer definition.

## Project Structure

Like the previous example, this project makes use of template inheritance, where there is a common 'project.yaml' file from which all of the configuratioins are derived.

The template provides the basic structure, with 'blocks' which may be substituted or extended by child templates. We use this functionaity in the configurations to override various components of the base configuraiton.

This is still a relatively simple project, as it does not reference any external template libraries. We will get to that in the next example.

## Code Generation

Note the output of the code generator. It has detected the inclusion of a dynamic import, thus it has automatically defined a function for importing dynamic modules.

Also note that it knows how to translate some of the rather clunky expreressions from the original YAML file, like calling a method, into relatively clean Python code.

---



#### Project Directory: "/home/dinalt/ai_assets/forgather/tutorials/project_gamma"

## Meta Config
Meta Config: [/home/dinalt/ai_assets/forgather/tutorials/project_gamma/meta.yaml](meta.yaml)

- [meta.yaml](meta.yaml)

Template Search Paths:
- [/home/dinalt/ai_assets/forgather/tutorials/project_gamma/templates](templates)

## Available Configurations
- [wider.yaml](templates/experiments/wider.yaml)
- [deeper.yaml](templates/experiments/deeper.yaml)
- [adam.yaml](templates/experiments/adam.yaml)
- [baseline.yaml](templates/experiments/baseline.yaml)

Default Configuration: baseline.yaml

Active Configuration: baseline.yaml

## Included Templates
- [experiments/baseline.yaml](templates/experiments/baseline.yaml)
    - [project.yaml](templates/project.yaml)
        - [formatting.yaml](templates/formatting.yaml)
### Config Metadata:

```python
{'citation': 'https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html',
 'config_class': 'fashion_trainer',
 'config_description': 'Base configuration, based on Torch tutorial '
                       'parameters.',
 'config_name': 'Fashion MNIST Trainer'}

```

## Modules
- [./src/trainer.py](src/trainer.py) : Trainer
    - [/home/dinalt/ai_assets/forgather/tutorials/project_gamma/./src/trainer.py](src/trainer.py) : trainer
## Output Targets
- meta
- main
- args
- model_params
- model
- training_data
- test_data
- train_dataloader
- test_dataloader
- loss_fn
- optimizer

## Preprocessed Config

```yaml

#---------------------------------------
#          Fashion MNIST Trainer         
#---------------------------------------
# 2024-08-16T04:31:35
# Description: Base configuration, based on Torch tutorial parameters.
# Project Dir: /home/dinalt/ai_assets/forgather/tutorials/project_gamma
# Citation: https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
#---------------------------------------


############# Config Vars ##############

# ns.hidden_dim = 512
# ns.epochs = 5
# ns.batch_size = 64
# ns.lr = 0.001
# ns.logging_steps = 100



########### Model Definition ###########

.define: &activation_fn !factory:torch.nn:ReLU@activation_fn []

.define: &model !singleton:torch.nn:Sequential@model
    - !factory:torch.nn:Flatten []
    - !factory:torch.nn:Linear [ 784, 512 ]
    - *activation_fn
    - !factory:torch.nn:Linear [ 512, 512 ]
    - *activation_fn
    - !factory:torch.nn:Linear [ 512, 10 ]

############### Dataset ################

.define: &transform !factory:torchvision.transforms:ToTensor@transform []

.define: &training_data !singleton:torchvision.datasets:FashionMNIST
    root: "data"
    train: True
    download: True
    transform: *transform

.define: &test_data !singleton:torchvision.datasets:FashionMNIST
    root: "data"
    train: False
    download: True
    transform: *transform

.define: &train_dataloader !singleton:torch.utils.data:DataLoader
    args: [ *training_data ]
    kwargs: { batch_size: 64 }

.define: &test_dataloader !singleton:torch.utils.data:DataLoader
    args: [ *test_data ]
    kwargs: { batch_size: 64 }

############### Trainer ################

.define: &model_params !singleton:call [ !singleton:getattr [ *model, "parameters" ] ]

# **Optimizer**

.define: &optimizer !singleton:torch.optim:SGD
    args: [ *model_params ]
    kwargs:
        lr: 0.001

# **Loss Function**

.define: &loss_fn !singleton:torch.nn:CrossEntropyLoss []

# **Trainer**

.define: &trainer !singleton:./src/trainer.py:Trainer@trainer
    train_dataloader: *train_dataloader
    test_dataloader: *test_dataloader
    model: *model
    loss_fn: *loss_fn
    optimizer: *optimizer
    epochs: 5
    batch_size: 64
    logging_steps: 100

################ Output ################

meta:
    config_name: "Fashion MNIST Trainer"
    config_description: "Base configuration, based on Torch tutorial parameters."
    config_class: "fashion_trainer"
    citation: "https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html"

main: *trainer

args:
    hidden_dim: 512
    epochs: 5
    batch_size: 64
    logging_steps: 100
    lr: 0.001
model_params: *model_params
model: *model
training_data: *training_data
test_data: *test_data
train_dataloader: *train_dataloader
test_dataloader: *test_dataloader
loss_fn: *loss_fn
optimizer: *optimizer

```

## Generated Code

```python
from torch.utils.data import DataLoader
from torch.nn import Sequential
from torch.nn import Linear
from torch.optim import SGD
from torch.nn import Flatten
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from torch.nn import CrossEntropyLoss
from torch.nn import ReLU
from importlib.util import spec_from_file_location, module_from_spec
import os
import sys

# Import a dynamic module.
def dynimport(module, name, searchpath):
    module_path = module
    module_name = os.path.basename(module).split(".")[0]
    module_spec = spec_from_file_location(
        module_name,
        module_path,
        submodule_search_locations=searchpath,
    )
    mod = module_from_spec(module_spec)
    sys.modules[module_name] = mod
    module_spec.loader.exec_module(mod)
    for symbol in name.split("."):
        mod = getattr(mod, symbol)
    return mod

Trainer = lambda: dynimport("./src/trainer.py", "Trainer", [])

def construct(
):
    transform = lambda: ToTensor()

    activation_fn = lambda: ReLU()

    model = Sequential(
        Flatten(),
        Linear(
            784,
            512,
        ),
        activation_fn(),
        Linear(
            512,
            512,
        ),
        activation_fn(),
        Linear(
            512,
            10,
        ),
    )

    trainer = Trainer()(
        train_dataloader=DataLoader(
            FashionMNIST(
                root='data',
                train=True,
                download=True,
                transform=transform(),
            ),
            batch_size=64,
        ),
        test_dataloader=DataLoader(
            FashionMNIST(
                root='data',
                train=False,
                download=True,
                transform=transform(),
            ),
            batch_size=64,
        ),
        model=model,
        loss_fn=CrossEntropyLoss(),
        optimizer=SGD(
            model.parameters(),
            lr=0.001,
        ),
        epochs=5,
        batch_size=64,
        logging_steps=100,
    )
    
    return trainer

```



## Construct Baseline Configuration

The "main" output is the model trainer, but we also get the model and the test-dataset as auxiliary outputs as a dictionary.

In [ ]:
from forgather.project import Project
import forgather.nb.notebooks as nb
from pprint import pp

# Load default baseline config
proj = Project()

outputs = proj(["main", "model", "test_data"])
pp(outputs)

# For easier access
trainer = outputs["main"]
model = outputs["model"]
test_data = outputs["test_data"]

### Examine Dataset and Predict

We can take a look at what's in the dataset using [Meerkat](http://meerkat.wiki/docs/start/tutorials/tutorial-data-frames.html).

The raw image data is a 32x32 tensor, which we can convert to a greyscale image for rendering.

The "label" is the is the ground-truth target the model is expected to predict.

We will also add a lambda to the DataPanel, which will get the model's prediction for the image. Without any training, the untrained model is expected to fail miserabl!

In [ ]:
import torch
import meerkat as mk
import torchvision.transforms as transforms
from PIL import Image

# Show target index key
print("Label Key: ", test_data.class_to_idx)

@torch.no_grad()
def predict(x):
    """
    Given a raw image, get the model's prediction.
    """
    # The image is stored as a 32x32 uint8 Tensor
    # Convert to float and add batch dimension
    model_input = (x / 255).unsqueeze(0)
    model.eval()

    # Get model's prediciton logits for input
    logits = model(model_input)

    # Get the index for the strongest prediction.
    return logits.argmax(1).item()

def make_datapanel():
    # Convert the test dataset's raw images into a Meerkat TensorColumn
    raw_img_column = mk.TensorColumn(test_data.data)
    
    dp = mk.DataPanel(
        {
            # Get label and convert to Python int
            "label": mk.TensorColumn(test_data.targets).defer(lambda x: x.item()),
            # Lazy model inference; get model's prediction from raw image
            "prediction": raw_img_column.defer(predict),
            # Lazy conversion of raw images to images
            "image": raw_img_column.defer(lambda x: Image.fromarray(x.numpy()))
        }
    )
    return dp

# Construct the Meerkat DataPanel and display the first 10 cells.
dp = make_datapanel()
dp[:10]()

## Train Model

The trainer is started by simply calling it.

In [ ]:
trainer()

### Test the Trained Model

We will display the same 10 data-cells as before, but now that the model has been trained, it's much less awful at the task.

In [ ]:
dp[:10]()

## Construct and Train with Adam Optimizer

This just goes straight from config to train, using the 'adam.yaml' configuration, where we have replaced the SGD optimizer with Adam.

Take a look at the actual config definition to see what changes were required to accomplish this.

In [ ]:
proj = Project(config_name="adam.yaml")
outputs = proj(["main", "model", "test_data"])

trainer = outputs["main"]
trainer()

# Regenerate the DataPanel and show predictions.
dp = make_datapanel()
dp[:10]()

## Run all Project Configurations

You can directly load the meta-config only and use it to find and iterate over all configurations in the project.

In [ ]:
from forgather.meta_config import MetaConfig

meta = MetaConfig()
for config_name, _ in meta.find_templates(meta.config_prefix):
    proj = Project(config_name=config_name)
    print(f"{ ' Starting ' + proj.config_name + ' ':-^60}")
    trainer = proj()
    trainer()